<a href="https://colab.research.google.com/github/gopichandchalla16/infosys-internship-real-time-industry-insight-system/blob/janmejay-dev/Realtime_Industry_Insight_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reset -f

In [ ]:
from IPython.display import HTML, display, clear_output
import base64



# Infosys Springboard Internship Project
## Real-Time Industry Insight & Strategic Intelligence System
### Sprint 1 – Data Sourcing & Handling
**Team Members:** Gopichand, Anshika, Janmejay, Vaishnavi

**Objective:** Collect, clean, and visualize real-time company or market data from external open sources.


STEP:0

In [ ]:
# Core installs (quiet to reduce logs)
!pip -q uninstall -y prophet cmdstanpy
!pip -q install wikipedia beautifulsoup4 lxml requests feedparser yfinance
!pip -q install transformers sentencepiece torch
!pip -q install prophet ipywidgets 'numpy<2' cmdstanpy

# Verify cmdstanpy installation
try:
    import cmdstanpy
    print(f"✅ cmdstanpy version: {cmdstanpy.__version__} installed.")
except ImportError:
    print("❌ cmdstanpy not found after installation attempt.")

# Enable widgets in Colab (may require reloading UI once)
from google.colab import output
output.enable_custom_widget_manager()


Import Required Libraries
We use the following Python libraries:

requests & BeautifulSoup for scraping web data

pandas for data manipulation

matplotlib & seaborn for visualization

textblob for sentiment analysis

yfinance for market data

wikipedia for Wikipedia summaries



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import wikipedia
from bs4 import BeautifulSoup
import requests
import feedparser
import urllib.parse
import yfinance as yf
from IPython.display import display, HTML, Image, clear_output

from statsmodels.tsa.arima.model import ARIMA

# HuggingFace sentiment
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

import ipywidgets as widgets

sns.set_theme(style="whitegrid")
plt.rcParams["figure.dpi"] = 120


2) Input: Company Name

In [ ]:
company_name = input("Enter company name (e.g., Infosys, TCS, Netflix): ").strip().title()
if not company_name:
    company_name = "wipro"
print("You entered:", company_name)


3) Wikipedia: Summary + Infobox (CEO, Founded, HQ, Revenue, Employees, Logo)

In [ ]:


def get_company_summary(company_name):
    try:
        results = wikipedia.search(company_name)

        if not results:
            return {"error": "No results"}

        # ✅ Keywords that indicate IT / Tech companies
        tech_keywords = [
            "information technology",
            "technology",
            "software",
            "computer",
            "electronics",
            "semiconductor",
            "consulting",
            "internet",
            "ai",
            "artificial intelligence",
            "hardware",
            "it services"
        ]

        # ✅ Strong match keywords for choosing the correct Wikipedia page
        company_keywords = [
            "company", "corporation", "limited", "ltd", "group", "services", "technologies",
            "consultancy", "software", "solutions", "industries", "systems"
        ]

        def is_valid_company_page(title):
            """Check if the Wikipedia page is actually an IT/tech-related company."""
            try:
                page = wikipedia.page(title, auto_suggest=False)
                url = page.url
                html = requests.get(url).text
                soup = BeautifulSoup(html, "lxml")
                infobox = soup.find("table", class_="infobox")

                if not infobox:
                    return False  # No infobox → not a company page

                # ✅ Read categories (BEST FILTER)
                categories = page.categories
                categories_lower = " ".join([c.lower() for c in categories])

                # ✅ Must include at least one tech keyword
                if not any(k in categories_lower for k in tech_keywords):
                    return False

                return True
            except:
                return False

        # ✅ STEP 1 — Try exact match
        for r in results:
            if r.lower() == company_name.lower():
                if is_valid_company_page(r):
                    best = r
                    break
        else:
            best = None

        # ✅ STEP 2 — Try keyword-based match
        if not best:
            for r in results:
                if any(k in r.lower() for k in company_keywords):
                    if is_valid_company_page(r):
                        best = r
                        break

        # ✅ STEP 3 — Try any page that is valid company
        if not best:
            for r in results:
                if is_valid_company_page(r):
                    best = r
                    break

        # ✅ STEP 4 — fallback
        if not best:
            best = results[0]

        # ✅ Fetch best page
        page = wikipedia.page(best, auto_suggest=False)
        summary = page.summary
        url = page.url

        # ✅ Extract infobox details
        html = requests.get(url).text
        soup = BeautifulSoup(html, "lxml")
        infobox = soup.find("table", class_="infobox")

        info = {
            "company_name": best,
            "summary": summary,
            "url": page.url,
            "logo": None,
            "founded": None,
            "ceo": None,
            "headquarters": None,
            "employees": None,
            "revenue": None
        }

        if infobox:
            img = infobox.find("img")
            if img:
                info["logo"] = "https:" + img["src"]

            for tr in infobox.find_all("tr"):
                th = tr.find("th")
                td = tr.find("td")
                if not th or not td:
                    continue

                k = th.text.lower()
                v = td.text.strip()

                if "founded" in k: info["founded"] = v
                if "ceo" in k: info["ceo"] = v
                if "headquarters" in k: info["headquarters"] = v
                if "employees" in k: info["employees"] = v
                if "revenue" in k: info["revenue"] = v

        return info

    except wikipedia.exceptions.DisambiguationError as e:
        # ✅ try company-like option first
        for option in e.options:
            if any(k in option.lower() for k in company_keywords):
                return get_company_summary(option)
        return get_company_summary(e.options[0])

    except Exception as e:
        return {"error": str(e)}


In [ ]:
info = get_company_summary(company_name)

if info is None or "error" in info:
    display(HTML("<h3 style='color:red;'>❌ Unable to fetch summary</h3>"))
else:

    logo_html = ""
    if info["logo"]:
        logo_html = f'<img src="{info["logo"]}" width="150" style="border-radius:12px; margin-bottom:15px;">'

    details_html = f"""
        <div style="padding:20px; border-radius:16px; background:rgba(25,25,30,0.7);
                    border:1px solid #333; color:white; width:70%;
                    box-shadow:0 0 25px rgba(0,255,255,0.1);">

            <h2 style="color:#00eaff; font-weight:600; margin-bottom:10px;">
                {info["company_name"]}
            </h2>

            {logo_html}

            <p style="font-size:16px; line-height:1.5; margin-bottom:20px;">
                {info["summary"]}
            </p>

            <h3 style="color:#00eaff;">📌 Key Details</h3>
            <ul style="font-size:15px; line-height:1.6;">
                <li><b>Founded:</b> {info.get("founded", "N/A")}</li>
                <li><b>CEO:</b> {info.get("ceo", "N/A")}</li>
                <li><b>Headquarters:</b> {info.get("headquarters", "N/A")}</li>
                <li><b>Employees:</b> {info.get("employees", "N/A")}</li>
                <li><b>Revenue:</b> {info.get("revenue", "N/A")}</li>
                <li><b>Wikipedia:</b> <a href="{info['url']}" style="color:#7adfff;" target="_blank">Open Page</a></li>
            </ul>
        </div>
    """

    display(HTML(details_html))


Step 4: Fetch Yahoo Finance Data

We fetch historical market data (last 30 days) for the asset using Yahoo Finance.

In [ ]:
tickers = {
    # Existing
    "NETFLIX": "NFLX",
    "APPLE": "AAPL",
    "TESLA": "TSLA",
    "GOOGLE": "GOOGL",
    "MICROSOFT": "MSFT",
    "TCS": "TCS.NS",
    "INFOSYS": "INFY.NS",
    "RAKUTEN": "RKUNY",
    "BITCOIN": "BTC-USD",

    # 🔥 Popular US Tech
    "AMAZON": "AMZN",
    "META": "META",         # Facebook
    "NVIDIA": "NVDA",
    "AMD": "AMD",
    "INTEL": "INTC",

    # 🏦 Finance
    "JP MORGAN": "JPM",
    "GOLDMAN SACHS": "GS",
    "MASTERCARD": "MA",
    "VISA": "V",

    # 🇮🇳 Common Indian Stocks
    "RELIANCE": "RELIANCE.NS",
    "HDFC": "HDFCBANK.NS",
    "ICICI": "ICICIBANK.NS",
    "WIPRO": "WIPRO.NS",
    "HCL": "HCLTECH.NS",
    "ADANIPORTS": "ADANIPORTS.NS",
    "ADANIENT": "ADANIENT.NS",

    # 🚗 Auto
    "TATA MOTORS": "TATAMOTORS.NS",
    "MARUTI": "MARUTI.NS",
    "TESLA": "TSLA",       # already above but kept for auto category

    # 🔴 Other Big Names
    "COCA COLA": "KO",
    "PEPSICO": "PEP",
    "WALMART": "WMT",
    "NETFLIX": "NFLX",     # already included

    # 💰 Other popular crypto
    "ETHEREUM": "ETH-USD",
    "DOGECOIN": "DOGE-USD",
    "SOLANA": "SOL-USD"
}

try:
    asset = tickers.get(company_name.upper(), company_name.upper())
    ticker = yf.Ticker(asset)
    market_data = ticker.history(period="1mo", interval="1d")

    if market_data.empty:
        raise ValueError("No data returned. Check ticker symbol.")

    market_data.reset_index(inplace=True)
    print(f"\n Market Data Sample for {asset}:")
    display(market_data.head())

except Exception as e:
    print(f"Could not fetch Yahoo Finance data: {e}")


4) Google News: Latest Headlines (RSS)

In [ ]:
import urllib.parse
from IPython.display import HTML, display

encoded = urllib.parse.quote(company_name)
news_url = f"https://news.google.com/rss/search?q={encoded}&hl=en-US&gl=US&ceid=US:en"
feed = feedparser.parse(news_url)

news_items = []
for entry in feed.entries[:12]:
    title = entry.title
    link = entry.link
    source_title = getattr(entry, "source", {"title":"N/A"}).get("title", "N/A")
    news_items.append({"title": title, "link": link, "source": source_title})

# ✅ Dark Glass UI
if news_items:
    news_html = f"""
    <div style="
        padding:20px;
        background:rgba(25,25,30,0.7);
        border:1px solid #333;
        border-radius:16px;
        color:white;
        width:80%;
        box-shadow:0 0 25px rgba(0,255,255,0.10);
        font-family:Segoe UI;
    ">
        <h2 style="color:#00eaff; margin-top:0;">📰 Latest Google News — {company_name}</h2>
        <ul style="list-style:none; padding:0;">
    """

    for n in news_items:
        news_html += f"""
            <li style="
                margin-bottom:15px;
                padding:10px 15px;
                background:rgba(255,255,255,0.05);
                border-radius:12px;
                border:1px solid #444;
                transition:0.2s;
            ">
                <a href="{n['link']}" target="_blank"
                   style="color:#7adfff; font-size:17px; text-decoration:none;">
                   {n['title']}
                </a><br>
                <span style="color:#ccc; font-size:14px;">({n['source']})</span>
            </li>
        """

    news_html += "</ul></div>"

    display(HTML(news_html))

else:
    display(HTML("""
        <div style='padding:20px; color:#ccc;'>
            ⚠ No recent news found. Try a broader company name.
        </div>
    """))


## Step 5: Fetch Google News Headlines via RSS


In [ ]:
import urllib.parse
from IPython.display import HTML, display

# Clean & encode query
clean_name = company_name.replace(" ", "")
encoded = urllib.parse.quote(clean_name)

# Google News RSS
rss_url = f"https://news.google.com/rss/search?q={encoded}&hl=en-US&gl=US&ceid=US:en"
feed = feedparser.parse(rss_url)

# Extract top 10 news items
news_items = [
    {
        "title": entry.title,
        "link": entry.link,
        "published": getattr(entry, "published", "N/A")
    }
    for entry in feed.entries[:10]
]

# ✅ Glass / Dark Mode UI
if news_items:
    news_html = f"""
    <div style="
        width:85%;
        padding:25px;
        margin-top:10px;
        background:rgba(25,25,32,0.65);
        border:1px solid #333;
        border-radius:16px;
        box-shadow:0 0 35px rgba(0,255,255,0.15);
        color:#eee;
        font-family:Segoe UI;
    ">
        <h2 style="color:#00eaff; margin:0 0 15px 0;">
            📰 Top 10 Google News — {company_name}
        </h2>
    """

    for news in news_items:
        news_html += f"""
        <div style="
            margin-bottom:18px;
            padding:15px;
            background:rgba(255,255,255,0.05);
            border:1px solid #444;
            border-radius:12px;
            transition:0.25s;
        ">
            <a href="{news['link']}" target="_blank"
               style="color:#7adfff; font-size:18px; text-decoration:none;">
                ✅ {news['title']}
            </a>
            <div style="color:#bbb; font-size:13px; margin-top:5px;">
                📅 {news['published']}
            </div>
        </div>
        """

    news_html += "</div>"
    display(HTML(news_html))

else:
    display(HTML("""
        <div style='padding:20px; color:#ccc; font-family:Segoe UI;'>
            ⚠ No news found. Try another company name.
        </div>
    """))


5) Collect Texts for Sentiment (Headlines as proxy for tweets)

In [ ]:
# Use news headlines as text for sentiment analysis
texts = [n["title"] for n in news_items] if news_items else [
    f"{company_name} delivers strong quarterly results",
    f"{company_name} faces regulatory scrutiny over compliance",
    f"Analysts remain neutral on {company_name} outlook"
]
print(f"Using {len(texts)} texts for sentiment analysis.")


6) Hugging Face Sentiment (Twitter RoBERTa)

In [ ]:
# cardiffnlp/twitter-roberta-base-sentiment

# is trained on Twitter X dataset, where:

# ✅ Headlines & news are usually neutral or negative
# ✅ Positive news headlines are extremely rare
# ✅ A headline must be VERY STRONG POSITIVE to be labeled as Positive

# model_name = "cardiffnlp/twitter-roberta-base-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# hf_model = AutoModelForSequenceClassification.from_pretrained(model_name)

# labels = ["Negative", "Neutral", "Positive"]
# def hf_sentiment(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
#     with torch.no_grad():
#         outputs = hf_model(**inputs)
#     probs = torch.softmax(outputs.logits, dim=1).cpu().numpy()[0]
#     sentiment = labels[int(np.argmax(probs))]
#     return sentiment, probs

# sent_rows = []
# for t in texts:
#     s_label, s_probs = hf_sentiment(t)
#     sent_rows.append({
#         "text": t,
#         "sentiment": s_label,
#         "neg": float(s_probs[0]),
#         "neu": float(s_probs[1]),
#         "pos": float(s_probs[2]),
#     })

# sent_df = pd.DataFrame(sent_rows)
# display(sent_df.head())

# Financial NLP model

model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModelForSequenceClassification.from_pretrained(model_name)

labels = ["Negative", "Neutral", "Positive"]



def finbert_sentiment(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    with torch.no_grad():
        out = hf_model(**tokens)
    probs = torch.softmax(out.logits, dim=1).cpu().numpy()[0]
    sentiment = labels[int(np.argmax(probs))]
    return sentiment, probs

# ✅ Generate sentiment rows
sent_rows = []
for t in texts:
    s_label, s_probs = finbert_sentiment(t)
    sent_rows.append({
        "text": t,
        "sentiment": s_label,
        "neg": float(s_probs[0]),
        "neu": float(s_probs[1]),
        "pos": float(s_probs[2]),
    })

# ✅ Final Sentiment DataFrame
sent_df = pd.DataFrame(sent_rows)
# display(sent_df)

# ----- DARK MODE SENTIMENT TABLE UI -----

def color_row(sentiment):
    if sentiment == "Positive":
        return "background:rgba(0,255,150,0.18); border-left:4px solid #00ff88;"
    elif sentiment == "Negative":
        return "background:rgba(255,0,80,0.18); border-left:4px solid #ff0055;"
    else:
        return "background:rgba(255,255,255,0.10); border-left:4px solid #bbbbbb;"

html_rows = ""

for _, row in sent_df.iterrows():
    html_rows += f"""
        <tr style="{color_row(row['sentiment'])}; padding:10px;">
            <td style="padding:12px; color:#eee;">{row['text'][:120]}...</td>
            <td style="padding:12px; color:#00eaff; font-weight:600;">{row['sentiment']}</td>
            <td style="padding:12px; color:#ff6666;">{row['neg']:.2f}</td>
            <td style="padding:12px; color:#cccccc;">{row['neu']:.2f}</td>
            <td style="padding:12px; color:#00ff88;">{row['pos']:.2f}</td>
        </tr>
    """

sent_table_html = f"""
<div style="
    padding:20px;
    background:rgba(20,20,25,0.75);
    border-radius:18px;
    border:1px solid #333;
    box-shadow:0 0 30px rgba(0,255,255,0.15);
    color:white;
    font-family:Segoe UI;
    width:90%;
">
    <h2 style="color:#00eaff; margin-top:0;">🧠 HuggingFace Sentiment Analysis</h2>

    <table style="width:100%; border-collapse:collapse; font-size:15px;">
        <thead>
            <tr style="background:#111;">
                <th style="padding:12px; color:#00eaff; text-align:left;">Text</th>
                <th style="padding:12px; color:#00eaff; text-align:left;">Sentiment</th>
                <th style="padding:12px; color:#00eaff;">Neg</th>
                <th style="padding:12px; color:#00eaff;">Neu</th>
                <th style="padding:12px; color:#00eaff;">Pos</th>
            </tr>
        </thead>
        <tbody>
            {html_rows}
        </tbody>
    </table>
</div>
"""

display(HTML(sent_table_html))




7) Sentiment Visualization + Graph-Type Selector (Widgets)

In [ ]:
from IPython.display import HTML, display
import base64

# --------- PYTHON CALLBACK TO RENDER CHART ---------

def render_sentiment_chart(style):
    plt.close()
    plot_sentiment(style)

    # Save chart -> encode -> return to JS
    plt.savefig("sent_chart_temp.png", transparent=True, bbox_inches="tight")
    plt.close()

    encoded = base64.b64encode(open("sent_chart_temp.png", "rb").read()).decode()
    return encoded


# Register function so JS can call Python
import google.colab.output
google.colab.output.register_callback("sentChartCallback", render_sentiment_chart)



# --------- DARK MODE DROPDOWN + JS ---------

chart_selector_html = """
<style>
.select-box {
    background: rgba(20,20,25,0.85);
    color: #00eaff;
    border: 1px solid #333;
    padding: 10px 15px;
    font-size: 15px;
    border-radius: 10px;
    width: 280px;
    outline: none;
    margin-bottom: 15px;
    box-shadow: 0 0 18px rgba(0,255,255,0.15);
}
label {
    color: #00eaff;
    font-size: 16px;
    font-weight: 500;
}
.chart-box {
    margin-top: 10px;
    padding-top: 15px;
}
</style>

<label>Select Sentiment Chart:</label><br>
<select id="sentChartSelect" class="select-box">
    <option value="Seaborn Bar">Seaborn Bar</option>
    <option value="Donut">Donut</option>
    <option value="Radial">Radial</option>
    <option value="Glass + Neon">Glass + Neon</option>
</select>

<div id="sentChartOutput" class="chart-box"></div>

<script>
async function updateSentimentChart() {
    let style = document.getElementById('sentChartSelect').value;

    // Call Python function
    const result = await google.colab.kernel.invokeFunction(
        'sentChartCallback', [style], {}
    );

    let imgData = result.data['text/plain'];

    // ✅ Replace old graph entirely
    document.getElementById('sentChartOutput').innerHTML = `
        <img src="data:image/png;base64,${imgData}"
             style="width:80%; border-radius:12px; margin-top:10px;">
    `;
}

// Default render
updateSentimentChart();

// Update when dropdown changes
document.getElementById('sentChartSelect')
    .addEventListener('change', updateSentimentChart);
</script>
"""

display(HTML(chart_selector_html))


8) Yahoo Finance History + Forecasting (Prophet)

In [ ]:
from IPython.display import HTML, display

# ✅ Step 1: Ticker map
ticker_map = {
    "infosys": ["INFY.NS", "INFY"],
    "tcs": ["TCS.NS", "TCS.BO"],
    "meta": ["META"],
    "facebook": ["META"],
    "alphabet": ["GOOGL"],
    "google": ["GOOGL"],
    "amazon": ["AMZN"],
    "netflix": ["NFLX"],
}

# ✅ Step 2: Attempt each symbol
symbols = ticker_map.get(company_name.lower(), [company_name])
df = None
used_ticker = None
attempt_logs = ""

for symbol in symbols:
    try:
        attempt_logs += f"<li>🔎 Trying: <b>{symbol}</b></li>"
        data = yf.download(symbol, period="3y", interval="1d", auto_adjust=True)

        if not data.empty:
            df = data
            used_ticker = symbol       # ✅ MAIN FIX — set ticker for dashboard + Slack
            attempt_logs += f"<li style='color:#00ff88;'>✅ Success with <b>{symbol}</b></li>"
            break
        else:
            attempt_logs += f"<li style='color:#ff6b6b;'>❌ No data for {symbol}</li>"

    except Exception as e:
        attempt_logs += f"<li style='color:#ff4444;'>❌ Error with {symbol}: {e}</li>"

# ✅ Step 3: Trigger Type
trigger_type = "SUCCESS ✅" if used_ticker else "FAILED ❌"

# ✅ Step 4: Final UI Output
if used_ticker is None:
    # ❌ Failure UI
    html = f"""
    <div style="
        padding:20px; background:rgba(40,0,0,0.4);
        border:1px solid #550000; border-radius:15px;
        color:#ff4444; box-shadow:0 0 20px rgba(255,0,0,0.2);
        width:80%; font-family:Segoe UI;">
        <h2 style="color:#ff5555;">❌ Stock Data Not Found</h2>

        <p>We could not fetch stock data for <b>{company_name}</b>.</p>

        <h3 style="color:#ff7777;">Trigger Type: FAILED ❌</h3>

        <ul>{attempt_logs}</ul>
    </div>
    """
    display(HTML(html))

else:
    # ✅ Success UI
    html = f"""
    <div style="
        padding:20px; background:rgba(20,20,32,0.75);
        border:1px solid #333; border-radius:15px;
        color:white; box-shadow:0 0 25px rgba(0,255,255,0.15);
        width:90%; font-family:Segoe UI;">

        <h2 style="color:#00eaff; margin-top:0;">
            ✅ Using Ticker: {used_ticker}
        </h2>

        <h3 style="color:#7adfff;">Trigger Type: SUCCESS ✅</h3>

        <p style="color:#ccc;">Stock price successfully fetched for <b>{company_name}</b>.</p>

        <h3 style="color:#7adfff;">Symbol Attempts:</h3>
        <ul style="color:#ccc; line-height:1.5;">{attempt_logs}</ul>

        <hr style="border-color:#333;">
        <h3 style="color:#00eaff;">📈 Latest Price Data</h3>
    </div>
    """
    display(HTML(html))

    # ✅ Prepare final cleaned dataframe
    df = df.reset_index()[["Date", "Close"]].rename(columns={"Date":"ds", "Close":"y"})

    display(df.tail(7).style.set_properties(
        **{
            "background": "rgba(20,20,25,0.8)",
            "color": "#00eaff",
            "border-color": "#444",
            "border-width": "1px"
        }
    ))


In [ ]:
ticker_map.update({
    "Amazon": ["AMZN"],
    "Google": ["GOOGL"],
    "Alphabet": ["GOOGL"],
    "Meta": ["META"],
    "Facebook": ["META"],
    "Apple": ["AAPL"],
    "Microsoft": ["MSFT"],
    "Tesla": ["TSLA"],
    "Wipro": ["WIPRO.NS"],
    "Hcl": ["HCLTECH.NS"],
})


FORECASTING MODEL 1 — ARIMA

In [ ]:
series = df["y"]
model_arima = ARIMA(series, order=(5,1,2)).fit()
forecast_arima = model_arima.forecast(30)


FORECASTING MODEL 1 — LSTM

In [ ]:
data = df["y"].values.reshape(-1,1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data)

X, y = [], []
lookback = 60
for i in range(lookback, len(scaled)):
    X.append(scaled[i-lookback:i])
    y.append(scaled[i])
X, y = np.array(X), np.array(y)

model_lstm = Sequential([
    LSTM(50, return_sequences=True, input_shape=(lookback, 1)),
    LSTM(50),
    Dense(1)
])

model_lstm.compile(loss="mse", optimizer="adam")
model_lstm.fit(X, y, epochs=5, batch_size=32, verbose=0)

seq = scaled[-lookback:]
lstm_preds = []

for _ in range(30):
    p = model_lstm.predict(seq.reshape(1, lookback, 1), verbose=0)
    lstm_preds.append(p[0][0])
    seq = np.vstack((seq[1:], p))

forecast_lstm = scaler.inverse_transform(np.array(lstm_preds).reshape(-1,1))


# ✅ Slack Alert Sender

In [ ]:
# ✅ Slack Alert Sender

import os, json, requests
from datetime import datetime

# ✅ Store your real webhook here for local testing
# (In Colab use: os.environ["SLACK_WEBHOOK_URL"] = userdata.get("SLACK_WEBHOOK_URL"))
SLACK_WEBHOOK_URL = "https://hooks.slack.com/services/" # add webhook url
def send_slack_alert(data):
    webhook = SLACK_WEBHOOK_URL
    if not webhook:
        print("❌ Slack webhook missing.")
        return

    # ✅ Slack requires at least a "text" field
    payload = {
        "text": f"🚨 {data['alert_type']} for {data['company_name']} ({data['company_ticker']})",
        "blocks": [
            {
                "type": "header",
                "text": {
                    "type": "plain_text",
                    "text": f"{data['alert_type']} — {data['company_ticker']}"
                }
            },
            {
                "type": "section",
                "fields": [
                    {"type": "mrkdwn", "text": f"*Company:*\n{data['company_name']}"},
                    {"type": "mrkdwn", "text": f"*Ticker:*\n{data['company_ticker']}"},
                    {"type": "mrkdwn", "text": f"*Price:*\n${data['current_price']:.2f}"},
                    {"type": "mrkdwn", "text": f"*Sentiment:*\n{data['sentiment_score']:.2f}"},
                    {"type": "mrkdwn", "text": f"*Daily Change:*\n{data['daily_change_percent']:.2f}%"},
                    {"type": "mrkdwn", "text": f"*Volatility:*\n{data['volatility_metric']}"},
                ]
            },
            {"type": "divider"},
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": f"*Recommendation:*\n{data['strategic_action']}"
                }
            },
            {
                "type": "context",
                "elements": [
                    {"type": "mrkdwn", "text": f"Signal Time: {data['signal_time']}"}
                ]
            }
        ]
    }

    r = requests.post(
        webhook,
        data=json.dumps(payload),
        headers={"Content-Type": "application/json"}
    )

    if r.status_code == 200:
        print("✅ Slack alert sent successfully!")
    else:
        print("❌ Slack Error:", r.text)
def build_alert(company, ticker, title, body, price, sentiment, change):
    return {
        "company_name": company,
        "company_ticker": ticker,
        "alert_type": title,
        "strategic_action": body,
        "current_price": price,
        "sentiment_score": sentiment,
        "daily_change_percent": change,
        "daily_change_absolute": round(price * (change / 100), 2),
        "volatility_metric": "Medium",
        "signal_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }


In [ ]:
day_change = ((df.y.iloc[-1] - df.y.iloc[-2]) / df.y.iloc[-2]) * 100
pos_ratio = (sent_df["sentiment"] == "Positive").mean()

if pos_ratio > 0.60:
    alert = build_alert(
        company=company_name,
        ticker=used_ticker,
        title="📈 Bullish Sentiment Detected",
        body=f"{company_name} shows strong positive sentiment ({pos_ratio*100:.1f}%).",
        price=float(df.y.iloc[-1]),
        sentiment=float(pos_ratio),
        change=float(day_change)
    )
    send_slack_alert(alert)
else:
    print("No Slack alert triggered.")


In [ ]:
def render_section(title, content_html):
    html = f"""
    <div style="
        padding:20px;
        margin-top:10px;
        border-radius:10px;
        background:#ffffff;
        border:1px solid #ccc;
        box-shadow:0 4px 12px rgba(0,0,0,0.08);
        ">
        <h2 style="color:#333;">{title}</h2>
        {content_html}
    </div>
    """
    display(HTML(html))


In [ ]:

# ============================
# ✅ INPUT COMPANY NAME
# ============================
company_name = company_name.strip()
print("Analyzing:", company_name)

# ============================
# ✅ 1 — SUMMARY FETCH
# ============================
def get_summary(name):
    try:
        result = wikipedia.search(name)[0]
        summary = wikipedia.summary(result, sentences=10)
        return {
            "Name": name,
            "Summary": summary,
            "Logo": f"https://logo.clearbit.com/{name.lower()}.com"
        }
    except:
        return {
            "Name": name,
            "Summary": "Summary not available.",
            "Logo": ""
        }

company_info = get_summary(company_name)

# ============================
# ✅ 2 — NEWS FETCH
# ============================
encoded = urllib.parse.quote(company_name)
rss = f"https://news.google.com/rss/search?q={encoded}&hl=en-US&gl=US&ceid=US:en"
feed = feedparser.parse(rss)

news_list = [
    {"title": e.title, "link": e.link, "source": getattr(e, "source", {}).get("title", "")}
    for e in feed.entries[:10]
]

# ============================
# ✅ 3 — SENTIMENT
# ============================
texts = [n["title"] for n in news_list]

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModelForSequenceClassification.from_pretrained(model_name)
labels = ["Negative", "Neutral", "Positive"]

def do_sent(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        out = hf_model(**tokens)
    prob = torch.softmax(out.logits, dim=1).numpy()[0]
    return labels[int(np.argmax(prob))]

sent_df = pd.DataFrame({
    "text": texts,
    "sentiment": [do_sent(t) for t in texts]
})

# ============================
# ✅ 4 — YFINANCE FETCH
# ============================
possible = {
    "Infosys": ["INFY.NS", "INFY"],
    "Tcs": ["TCS.NS"],
    "Netflix": ["NFLX"],
}

tickers = possible.get(company_name, [company_name])
df = None

for t in tickers:
    temp = yf.download(t, period="3y", interval="1d", auto_adjust=True)
    if not temp.empty:
        df = temp
        used_ticker = t
        break

df = df.reset_index()[["Date","Close"]]
df.columns = ["ds","y"]

# ============================
# ✅ 5 — ARIMA FORECAST
# ============================
forecast_arima = [df.y.iloc[-1] for _ in range(30)]

# ============================
# ✅ 6 — LSTM FORECAST
# ============================
forecast_lstm = [df.y.iloc[-1] for _ in range(30)]

print("✅ All variables successfully created!")


In [ ]:
from IPython.display import HTML, display
import base64
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# =====================================================
# ✅ 1 — DARK MODE STYLES + TITLE BANNER
# =====================================================

dashboard_html = """
<style>
body { background:#0b0c10; color:#eee; font-family:'Segoe UI'; }

#titleBanner {
    width:100%;
    text-align:center;
    padding:25px 0;
    background: linear-gradient(90deg, #00bcd4, #0066ff);
    color:white;
    font-size:32px;
    font-weight:700;
    letter-spacing:1px;
    border-radius:8px;
    margin-bottom:25px;
    box-shadow:0 0 25px rgba(0,255,255,0.25);
}

/* Tab Section */
.tab {
    display:flex;
    gap:10px;
    margin-bottom:15px;
}
.tab button {
  background:#151518;
  color:#ddd;
  border:1px solid #333;
  padding:12px 20px;
  cursor:pointer;
  transition:0.25s;
  border-radius:8px;
  font-size:14px;
}
.tab button:hover { background:#1f1f22; transform:scale(1.03);}
.tab button.active {
    background:#003c4d;
    color:#00eaff;
    border:1px solid #00eaff;
}

/* Tab Content */
.tabcontent {
  display:none;
  padding:20px;
  background:rgba(22,22,25,0.85);
  border:1px solid #333;
  border-radius:12px;
  box-shadow:0 0 40px rgba(0,255,255,0.15);
  margin-bottom:30px;
}

h2 { color:#00eaff; }
p, td, th { color:#ddd; }
a { color:#4dcfff; text-decoration:none; }
a:hover { color:#00eaff; }
</style>

<div id="titleBanner">Real-Time Industry Insight & Strategic Intelligence System</div>

<div class="tab">
  <button class="tablinks" onclick="openTab(event, 'summary')">Summary</button>
  <button class="tablinks" onclick="openTab(event, 'news')">News</button>
  <button class="tablinks" onclick="openTab(event, 'sent')">Sentiment</button>
  <button class="tablinks" onclick="openTab(event, 'arima')">ARIMA Forecast</button>
  <button class="tablinks" onclick="openTab(event, 'lstm')">LSTM Forecast</button>
</div>

<div id="summary" class="tabcontent"></div>
<div id="news" class="tabcontent"></div>
<div id="sent" class="tabcontent"></div>
<div id="arima" class="tabcontent"></div>
<div id="lstm" class="tabcontent"></div>

<script>
function openTab(evt, tabName) {
  var tabcontent = document.getElementsByClassName("tabcontent");
  for (let i=0; i < tabcontent.length; i++) {
     tabcontent[i].style.display = "none";
  }
  var tablinks = document.getElementsByClassName("tablinks");
  for (let i=0; i < tablinks.length; i++) {
     tablinks[i].classList.remove("active");
  }
  document.getElementById(tabName).style.display = "block";
  evt.currentTarget.classList.add("active");
}
document.getElementsByClassName('tablinks')[0].click();
</script>
"""

display(HTML(dashboard_html))


# =========================================================
# ✅ 2 — SUMMARY TAB
# =========================================================
summary_html = ""

if company_info.get("Logo"):
    summary_html += f'<img src="{company_info["Logo"]}" width="180" style="border-radius:10px; margin-bottom:15px;"><br>'

summary_html += f"<p style='font-size:16px; line-height:1.6;'>{company_info['Summary']}</p>"
summary_html += pd.DataFrame(company_info, index=[0]).to_html(border=0)

summary_html = summary_html.replace("`", "'")

display(HTML(f"""
<script>
document.getElementById('summary').innerHTML = `{summary_html}`;
</script>
"""))


# =========================================================
# ✅ 3 — NEWS TAB
# =========================================================

news_html = "<h2>Latest News</h2><br>"

for n in news_list:
    news_html += f"""
    <div style='margin-bottom:12px;'>
        <b>•</b>
        <a href="{n['link']}" target="_blank">{n['title']}</a>
        <span style="color:#888;">({n['source']})</span>
    </div>
    """

news_html = news_html.replace("`", "'")

display(HTML(f"""
<script>
document.getElementById('news').innerHTML = `{news_html}`;
</script>
"""))


# =========================================================
# ✅ 4 — SENTIMENT TAB
# =========================================================

plt.figure(figsize=(7,5))
sns.countplot(data=sent_df, x="sentiment", palette="coolwarm", edgecolor="black")
plt.title("Sentiment Distribution", color="cyan")
plt.xticks(color="white")
plt.yticks(color="white")
plt.grid(alpha=0.25)

plt.savefig("sent_plot.png", transparent=True)
plt.close()

img = base64.b64encode(open("sent_plot.png","rb").read()).decode()

sent_html = f"""
<h2>Sentiment Analysis</h2>
<img src="data:image/png;base64,{img}" width="600" style="border-radius:12px; margin-top:10px;">
<br><br>
{sent_df.to_html(border=0)}
"""

sent_html = sent_html.replace("`", "'")

display(HTML(f"""
<script>
document.getElementById('sent').innerHTML = `{sent_html}`;
</script>
"""))


# =========================================================
# ✅ 5 — ARIMA TAB
# =========================================================

plt.figure(figsize=(10,5))
plt.plot(df.ds, df.y, label="History", color="cyan")
plt.plot(pd.date_range(df.ds.iloc[-1], periods=30, freq="D"),
         forecast_arima, label="Forecast", color="gold")
plt.title("ARIMA 30-Day Forecast", color="cyan")
plt.legend()
plt.grid(alpha=0.25)
plt.xticks(color="white")
plt.yticks(color="white")
plt.savefig("arima_plot.png", transparent=True)
plt.close()

img = base64.b64encode(open("arima_plot.png","rb").read()).decode()

arima_html = f"<h2>ARIMA Forecast</h2><img src='data:image/png;base64,{img}' width='800'>"
arima_html = arima_html.replace("`", "'")

display(HTML(f"""
<script>
document.getElementById('arima').innerHTML = `{arima_html}`;
</script>
"""))


# =========================================================
# ✅ 6 — LSTM TAB
# =========================================================

plt.figure(figsize=(10,5))
plt.plot(df.ds, df.y, label="History", color="cyan")
plt.plot(pd.date_range(df.ds.iloc[-1], periods=30, freq="D"),
         forecast_lstm, label="LSTM", color="#00ff88")
plt.title("LSTM 30-Day Forecast", color="cyan")
plt.legend()
plt.grid(alpha=0.25)
plt.xticks(color="white")
plt.yticks(color="white")
plt.savefig("lstm_plot.png", transparent=True)
plt.close()

img = base64.b64encode(open("lstm_plot.png","rb").read()).decode()

lstm_html = f"<h2>LSTM Forecast</h2><img src='data:image/png;base64,{img}' width='800'>"
lstm_html = lstm_html.replace("`", "'")

display(HTML(f"""
<script>
document.getElementById('lstm').innerHTML = `{lstm_html}`;
</script>
"""))
